#### Audio
- 음성 비서
    - AI 콜센터, 음성 챗봇, 자동차 음성 비서
    - speech-to-speech(realtime)
- Text-to-Speech(텍스트를 사람 목소리로 변환)
-Speech-to-Text(음성을 텍스트로 변환)
- Response API 는 지원안함
- realtime API(audio → 둘다), Transcription API(audio → text), Speech API(text → audio) 사용
chat.completions(audio,text → audio,text)

In [1]:
from dotenv import load_dotenv, find_dotenv
# .env 를 환경변수로 설정
load_dotenv(find_dotenv())

True

In [2]:
import base64
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-audio",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": "Is a golden retriever a good family dog?"
        }
    ]
)

print(completion.choices[0])

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("./audio/dog.wav", "wb") as f:
    f.write(wav_bytes)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=ChatCompletionAudio(id='audio_69981515d23c8191875e43f9041f0498', data='...', expires_at=1771578149, transcript='Absolutely! Golden Retrievers are known for being excellent family dogs. They are friendly, gentle, and great with kids. They’re also intelligent, easy to train, and generally get along well with other pets. Plus, they have a very loyal and loving nature, making them a wonderful addition to many households.'), function_call=None, tool_calls=None))


In [4]:
# text -> 음성
speech = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="ash",
    input="안녕하세요"
)

with open("./audio/tts.wav","wb") as f:
    f.write(speech.read())

In [5]:
# 음성 -> 텍스트
with open("./audio/obama.mp3","rb") as f:
    transcript = client.audio.transcriptions.create(
        model="gpt-4o-mini-transcribe",
        file=f
    )

print(transcript.text)

Hello, Chicago! It's good to be home. Tonight, it's my turn to say thanks. And every day I have learned from you. You made me a better president. And you made me a better man. I can't do that. Where I learned that change only happens when ordinary people get involved and they get engaged. And they come together to demand it. After eight years as your president, I still believe that. In 10 days, the world will witness a hallmark of our democracy. No, no, no, no, no. The peaceful transfer of power from one freely elected president to the next. I committed to President-Elect Trump that my administration would ensure the smoothest possible transition, just as President Bush did for me. Because, after all, we remain the wealthiest, most powerful, and most respected nation on Earth. Michelle, for the past 25 years, you have not only been my wife and mother of my children, you have been my best friend. You have made me proud and you have made the country proud. My fellow Americans, it has bee

In [ ]:
import base64
import requests
from openai import OpenAI

client = OpenAI()

# Fetch the audio file and convert it to a base64 encoded string
url = "https://cdn.openai.com/API/docs/audio/alloy.wav"
response = requests.get(url)
# 잘되는지 검증
response.raise_for_status()
wav_data = response.content
encoded_string = base64.b64encode(wav_data).decode('utf-8')

completion = client.chat.completions.create(
    model="gpt-audio",
    # 응답구조 : textg + audio
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": [
                { 
                    "type": "text",
                    "text": "What is in this recording?"
                },
                {
                    "type": "input_audio",
                    "input_audio": {
                        "data": encoded_string,
                        "format": "wav"
                    }
                }
            ]
        },
    ]
)

print(completion.choices[0].message)

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("./audio/dog.wav", "wb") as f:
    f.write(wav_bytes)
# 오디오 -> 오디오를 기반으로 질문을 함 -> 답변 :텍스트와 오디오
# 답변으로 받은 텍스트 출력
print(f"답변 {completion.choices[0].message.audio.transcript}")
#답변으로 받은 오디오 출력
audio_base64 = completion.choices[0].message.audio.data
audio_bytes = base64.b64decode(audio_base64)

with open("./audio/reponse.wav","wb") as fw:
    fw.write(audio_bytes)